In [181]:
from bs4 import BeautifulSoup
import requests
import urllib.request
import time
import itertools
import pandas as pd
import random


In [44]:
def search_by(transaction_type, location, property_type, radius = 0, price_min = '', price_max = '',min_beds = '',max_beds = '',surface_min = '', surface_max = ''):
    '''
    function for set the filters applied
    '''
    wortimmo = str("https://www.wortimmo.lu/en/search?property_search_engine%5BtransactionType%5D="+transaction_type
    +"&property_search_engine%5Blocation%5D="+location
    +"&property_search_engine%5Bradius%5D="+str(radius)
    +"&property_search_engine%5BpropertyTypes%5D%5B%5D="+str(property_type)
    +"&property_search_engine%5BpurchasePriceMin%5D="+str(price_min)
    +"&property_search_engine%5BpurchasePriceMax%5D="+str(price_max)
    +"&property_search_engine%5BbedroomMin%5D="+str(min_beds)
    +"&property_search_engine%5BbedroomMax%5D="+str(max_beds)
    +"&property_search_engine%5BsurfaceMin%5D="+str(surface_min)
    +"&property_search_engine%5BsurfaceMax%5D="+str(surface_max)
    +"&property_search_engine%5Bsubmit%5D=&sort_by=newest_date")
    return wortimmo
            

In [45]:
search_by(transaction_type='vente', location='country-1', property_type=0)

'https://www.wortimmo.lu/en/search?property_search_engine%5BtransactionType%5D=vente&property_search_engine%5Blocation%5D=country-1&property_search_engine%5Bradius%5D=0&property_search_engine%5BpropertyTypes%5D%5B%5D=0&property_search_engine%5BpurchasePriceMin%5D=&property_search_engine%5BpurchasePriceMax%5D=&property_search_engine%5BbedroomMin%5D=&property_search_engine%5BbedroomMax%5D=&property_search_engine%5BsurfaceMin%5D=&property_search_engine%5BsurfaceMax%5D=&property_search_engine%5Bsubmit%5D=&sort_by=newest_date'

In [170]:
headers = ({'User-Agent':
            'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'})

wortimmo = search_by(transaction_type='vente', location='country-1', property_type=0, min_beds=2)
response = requests.get(wortimmo, headers=headers)
html_page = BeautifulSoup(response.text, 'html.parser')



In [230]:
house_containers = html_page.find_all('div', class_="c-organism c-property-result-block")
len(house_containers)
a=0
for container in house_containers[0:5]:
    description = container.find_all(class_="agency")[0]
    a = description.a['title']
print(a)

AFIL IMMO S.A.


In [165]:
from PIL import Image
from io import StringIO ## for Python 3



url = img['big-src']
response = requests.get(url)
import urllib2

im = Image.open(urllib2.urlopen(url))

ModuleNotFoundError: No module named 'urllib2'

In [235]:
def get_data(html_page,num):

    # setting up the lists that will form our dataframe with all the results
    descriptions = []
    prices = []
    areas = []
    places = []
    rooms = []
    parkings = []
    agencies = []
    urls = []

    house_containers = html_page.find_all('div', class_="c-organism c-property-result-block")

    if house_containers != []:
        for container in house_containers[0:num]: 
            # Description
            description = container.find_all(class_="c-title")[0].text
            description = description.replace("\n    ","")
            descriptions.append(description)

            # Price          
            price = container.find_all('div', class_="c-price c-text c-text__price")[0].text
            price = price.replace("\n    ","")
            price = price.replace("\n","")
            currency = price[-1]
            price = float(price.replace(" ","")[:-1])
            prices.append(price)

            # Area
            area = container.find_all('span')[4].text
            area = area.replace("\n                ","")
            area = area.replace("\n            ","")
            measure = area[-2:]
            area = "".join(itertools.takewhile(str.isdigit, area))
            areas.append(area)            

            # Rooms
            num_hab = container.find_all('span')[5].text   
            num_hab = num_hab.replace("\n                ","")
            num_hab = "".join(itertools.takewhile(str.isdigit, num_hab))
            rooms.append(num_hab)

            # place
            place = description.split("in",1)[1]
            places.append(place)

            # Parking
            num_parking = container.find_all('span')[6].text
            num_parking = num_parking.replace("\n                ","")
            num_parking = num_parking.replace("\n        ","")
            if num_parking == '-':
                num_parking = 0
            parkings.append(int(num_parking))
            
            # Agency
            agency = container.find_all(class_="agency")[0]
            agency = agency.a['title']
            agencies.append(agency)

            # url
            link = 'https://www.wortimmo.lu' + container.find_all('a')[0].get('href')
            urls.append(link)

            # image
#             img = str(container.find_all('img')[0])
#             img = img['big-src']
#             thumbnails.append(img)

            # Create dataframe
        cols = ['Description', str('Price ('+currency+')'), str('Area ('+measure+')'), 'Num. Rooms', 'Zone', 'Num. Parkings', 'Agency', 'URL']

        wortimmo_df = pd.DataFrame({'Description': descriptions,
                                   str('Price ('+currency+')'): price,
                                   str('Area ('+measure+')'): areas,
                                   'Num. Rooms': rooms,
                                   'Num. Parkings': parkings,
                                   'Agency': agencies,
                                   'Zone': places,
                                   'URL': urls,
                                  })[cols]
        return wortimmo_df
        print ('Dataframe created')
    else:
        return"No results found"            

    
    






In [236]:
get_data(html_page,5)


,Description,Price (€),Area (m²),Num. Rooms,Zone,Num. Parkings,Agency,URL
0,Apartment with 2 bedroom(s) to sell in Bridel,480000.0,90,2,Bridel,0,New Keys Sàrl,https://www.wortimmo.lu/en/buy/apartment/ouest...
1,Apartment with 2 bedroom(s) to sell in Dondelange,480000.0,130,2,Dondelange,0,New Keys Sàrl,https://www.wortimmo.lu/en/buy/apartment/ouest...
2,Apartment with 2 bedroom(s) to sell in Itzig,480000.0,68,2,Itzig,1,New Keys Sàrl,https://www.wortimmo.lu/en/buy/apartment/centr...
3,Apartment with 2 bedroom(s) to sell in Luxembo...,480000.0,83,2,Luxembourg-Kirchberg,0,New Keys Sàrl,https://www.wortimmo.lu/en/buy/apartment/centr...
4,Apartment with 2 bedroom(s) to sell in Niederkorn,480000.0,70,2,Niederkorn,2,AFIL IMMO S.A.,https://www.wortimmo.lu/en/buy/apartment/sud/n...
